In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from demo_classes import workspace, Vessel

The workspace contains vessels. There is one workspace per experiment. 

In [3]:
workspace

Workspace object containing vessels []

Lets start by creating some vessels to go in the workspace

In [4]:
GBL_vessel = Vessel(name="GBL_Vessel")
EthNH3I_vessel = Vessel(name="EthNH3I_Vessel")
PbI2_vessel = Vessel(name="PbI2_Vessel")

In [5]:
for v in GBL_vessel, EthNH3I_vessel, PbI2_vessel:
    workspace.add(v)

Whats the state of our workspace now? 

This is just the contents of the vessels and their temperatures right now

In an escalate experiment, this will take on different values after each action. 

In [6]:
workspace.state

{'GBL_Vessel': {'solvents': {}, 'solutes': {}, 'temp': None},
 'EthNH3I_Vessel': {'solvents': {}, 'solutes': {}, 'temp': None},
 'PbI2_Vessel': {'solvents': {}, 'solutes': {}, 'temp': None}}

Now lets create some materials! 

In [7]:
from demo_classes import MaterialModel

In [8]:
GBL = MaterialModel('GBL', 
                    {'molar_mass': 86.0891}, 
                   state='liquid')

PbI2 = MaterialModel(name='PbI2', 
                     properties={'molecular_weight': 461.01}, 
                    state='solid')

EthNH3I = MaterialModel(name='Ethylammonium Iodide', 
                        properties={'molecular_weight': 173}, 
                       state='solid')

In [9]:
GBL_vessel.add({GBL: 1}) # add one unit of GBL to it 

In [10]:
workspace.state

{'GBL_Vessel': {'solvents': {MaterialModel of GBL: 1},
  'solutes': {},
  'temp': None},
 'EthNH3I_Vessel': {'solvents': {}, 'solutes': {}, 'temp': None},
 'PbI2_Vessel': {'solvents': {}, 'solutes': {}, 'temp': None}}

In [11]:
EthNH3I_vessel.add({EthNH3I: 1})
PbI2_vessel.add({PbI2: 1})

We probably want all materials to have a state of matter so we can just call .add()
rather than add_{solute,solvent}. Solids and liquids will be added to appropriate dict

In [12]:
workspace.state

{'GBL_Vessel': {'solvents': {MaterialModel of GBL: 1},
  'solutes': {},
  'temp': None},
 'EthNH3I_Vessel': {'solvents': {},
  'solutes': {MaterialModel of Ethylammonium Iodide: 1},
  'temp': None},
 'PbI2_Vessel': {'solvents': {},
  'solutes': {MaterialModel of PbI2: 1},
  'temp': None}}

An initial state for the experiment should be specified as part of the experiment setup  
(i.e., I have x_i amount of material i in vessel j, for all i, j)

Then the user defines a list of actions. After each action is executed, they mutate the state according to some rules. We store the list of mutated states, these are like a movie of the experiment.

So lets define some actions

In [13]:
from demo_classes import ActionBaseClass

Shekar: Workflow can take a workspace as a parameter, workflow can be the state machine

These class definitions are like action_defs, or action_templates if you prefer:

In [14]:
class Transfer(ActionBaseClass): 
    
    """We'll really want a liquid transfer and a solid transfer that take appropriate units
    (volume, mass)
    """
    
    def __init__(self, source=None, dest=None,amount=0): 
        """The action parameter is volume
        All actions need a source and a dest, which are... Vessels? Materials? 
        Lets go with Vessels right now for convenience, can revisit
        """
        
        self.amount = amount
        self.source = source
        self.dest   = dest
        
    def do(self): 
        self.dest.add({self.source.remove(self.amount): self.amount})
        
    def undo(self): 
        """Might not need this"""
        self.source.add({self.dest.remove(self.amount): self.amount})

In [15]:
workspace.state

{'GBL_Vessel': {'solvents': {MaterialModel of GBL: 1},
  'solutes': {},
  'temp': None},
 'EthNH3I_Vessel': {'solvents': {},
  'solutes': {MaterialModel of Ethylammonium Iodide: 1},
  'temp': None},
 'PbI2_Vessel': {'solvents': {},
  'solutes': {MaterialModel of PbI2: 1},
  'temp': None}}

In [16]:
stockA_vessel = Vessel(name="Stock A Vessel")

In [17]:
workspace.add(stockA_vessel)

In [18]:
workspace.state

{'GBL_Vessel': {'solvents': {MaterialModel of GBL: 1},
  'solutes': {},
  'temp': None},
 'EthNH3I_Vessel': {'solvents': {},
  'solutes': {MaterialModel of Ethylammonium Iodide: 1},
  'temp': None},
 'PbI2_Vessel': {'solvents': {},
  'solutes': {MaterialModel of PbI2: 1},
  'temp': None},
 'Stock A Vessel': {'solvents': {}, 'solutes': {}, 'temp': None}}

In [19]:
from decimal import Decimal

In [20]:
import decimal

In [21]:
decimal.getcontext().prec = 2

In [22]:
make_stock_A = [Transfer(EthNH3I_vessel, stockA_vessel, Decimal(.1)), 
                Transfer(PbI2_vessel, stockA_vessel, Decimal(.2)), 
                Transfer(GBL_vessel, stockA_vessel, Decimal(.6)), 
               ]

In [23]:
#%pdb

In [24]:
from pprint import pprint

(need to deal with floating point rounding errors) 

In [25]:
for step in make_stock_A: 
    step.do()
    pprint(workspace.state)
    print('\n=========\n')

{'EthNH3I_Vessel': {'solutes': {MaterialModel of Ethylammonium Iodide: Decimal('0.8999999999999999944488848769')},
                    'solvents': {},
                    'temp': None},
 'GBL_Vessel': {'solutes': {},
                'solvents': {MaterialModel of GBL: 1},
                'temp': None},
 'PbI2_Vessel': {'solutes': {MaterialModel of PbI2: 1},
                 'solvents': {},
                 'temp': None},
 'Stock A Vessel': {'solutes': {MaterialModel of Ethylammonium Iodide: Decimal('0.1000000000000000055511151231')},
                    'solvents': {},
                    'temp': None}}


{'EthNH3I_Vessel': {'solutes': {MaterialModel of Ethylammonium Iodide: Decimal('0.8999999999999999944488848769')},
                    'solvents': {},
                    'temp': None},
 'GBL_Vessel': {'solutes': {},
                'solvents': {MaterialModel of GBL: 1},
                'temp': None},
 'PbI2_Vessel': {'solutes': {MaterialModel of PbI2: Decimal('0.7999999999999999888977

Takeaway: vessel has properties, actions manipulate them. 